In [3]:
from curses.ascii import isspace
%load_ext autoreload
%autoreload 2


import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'
import keras
import numpy as np
from graph import *
from encoder import create_encoder
import pandas as pd

from data_processing import processer
from semantic_clustering import *

from RepresentationLearner import RepresentationLearner
from compute import *
from GNN import *
import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:


num_classes = 10
input_shape = (32, 32, 3)

(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()


In [5]:
x_data = np.concatenate([x_train, x_test])
y_data = np.concatenate([y_train, y_test])

classes = [
    'airplane',
    'automobile',
    'bird',
    'cat',
    'deer',
    'dog',
    'frog',
    'horse',
    'ship',
    'truck',
]

In [7]:
target_size = 32
representation_dims = 512
projection_units = 128
num_clusters = 28
kn = 5
tune_encoder_during_clustering = False

In [8]:
x_data = x_data.astype('float32')
processer.layers[-1].adapt(x_data)

array([[[[ 59.,  62.,  63.],
         [ 43.,  46.,  45.],
         [ 50.,  48.,  43.],
         ...,
         [158., 132., 108.],
         [152., 125., 102.],
         [148., 124., 103.]],

        [[ 16.,  20.,  20.],
         [  0.,   0.,   0.],
         [ 18.,   8.,   0.],
         ...,
         [123.,  88.,  55.],
         [119.,  83.,  50.],
         [122.,  87.,  57.]],

        [[ 25.,  24.,  21.],
         [ 16.,   7.,   0.],
         [ 49.,  27.,   8.],
         ...,
         [118.,  84.,  50.],
         [120.,  84.,  50.],
         [109.,  73.,  42.]],

        ...,

        [[208., 170.,  96.],
         [201., 153.,  34.],
         [198., 161.,  26.],
         ...,
         [160., 133.,  70.],
         [ 56.,  31.,   7.],
         [ 53.,  34.,  20.]],

        [[180., 139.,  96.],
         [173., 123.,  42.],
         [186., 144.,  30.],
         ...,
         [184., 148.,  94.],
         [ 97.,  62.,  34.],
         [ 83.,  53.,  34.]],

        [[177., 144., 116.],
       

In [9]:
encoder = create_encoder(
    representation_dims
)

In [10]:
representation_learner = RepresentationLearner(encoder, projection_units, num_augmentations=4)

lr_scheduler = keras.optimizers.schedules.CosineDecay(
    initial_learning_rate=0.001,
    decay_steps=500,
    alpha=0.1
)
optimizer = keras.optimizers.AdamW(learning_rate=lr_scheduler, weight_decay=0.0001)

representation_learner.compile(
    optimizer=optimizer,
    jit_compile=False,
)

history = representation_learner.fit(
    x=x_data,
    batch_size=512,
    epochs=50
)



Epoch 1/50


I0000 00:00:1748596135.843657    1778 cuda_dnn.cc:529] Loaded cuDNN version 90501


118/118 ━━━━━━━━━━━━━━━━━━━━ 110s 573ms/step - loss: 29.0362
Epoch 2/50
118/118 ━━━━━━━━━━━━━━━━━━━━ 63s 536ms/step - loss: 28.2116
Epoch 3/50
118/118 ━━━━━━━━━━━━━━━━━━━━ 63s 536ms/step - loss: 27.9178
Epoch 4/50
118/118 ━━━━━━━━━━━━━━━━━━━━ 64s 542ms/step - loss: 27.7095
Epoch 5/50
118/118 ━━━━━━━━━━━━━━━━━━━━ 64s 544ms/step - loss: 27.6018
Epoch 6/50
118/118 ━━━━━━━━━━━━━━━━━━━━ 61s 521ms/step - loss: 27.5573
Epoch 7/50
118/118 ━━━━━━━━━━━━━━━━━━━━ 64s 546ms/step - loss: 27.5222
Epoch 8/50
118/118 ━━━━━━━━━━━━━━━━━━━━ 64s 546ms/step - loss: 27.4922
Epoch 9/50
118/118 ━━━━━━━━━━━━━━━━━━━━ 64s 543ms/step - loss: 27.4728
Epoch 10/50
118/118 ━━━━━━━━━━━━━━━━━━━━ 65s 552ms/step - loss: 27.4525
Epoch 11/50
118/118 ━━━━━━━━━━━━━━━━━━━━ 62s 523ms/step - loss: 27.4290
Epoch 12/50
118/118 ━━━━━━━━━━━━━━━━━━━━ 67s 570ms/step - loss: 27.4124
Epoch 13/50
118/118 ━━━━━━━━━━━━━━━━━━━━ 88s 545ms/step - loss: 27.3946
Epoch 14/50
118/118 ━━━━━━━━━━━━━━━━━━━━ 65s 548ms/step - loss: 27.3822
Epoch 15/50

In [15]:
batch_size = 500
feature_vectors = encoder.predict(x_data, batch_size=batch_size, verbose=1)
feature_vectors = keras.utils.normalize(feature_vectors)


I0000 00:00:1748600128.484691    1776 service.cc:152] XLA service 0x7f69a4c303c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1748600128.484726    1776 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2025-05-30 11:15:28.529490: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-05-30 11:15:29.551994: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1483', 4 bytes spill stores, 4 bytes spill loads

2025-05-30 11:15:29.559885: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1483', 3312 bytes spill stores, 3016 bytes spill loads

2025-05-30 11:15:29.595081: I ext

 13/120 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step

I0000 00:00:1748600132.928119    1776 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


120/120 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step


In [16]:
knns = compute_knn(feature_vectors, batch_size=batch_size, kn=kn)


100%|██████████| 120/120 [00:04<00:00, 25.38it/s]


In [ ]:
for layer in encoder.layers:
    layer.trainable = tune_encoder_during_clustering

clustering_model = create_clustering_model(encoder, num_clusters, input_shape, name='clustering')
clustering_learner = create_clustering_learner(clustering_model, input_shape)

losses = [ClustersEntropyLoss(), ClustersEntropyLoss(entropy_loss_weight=5)]

inputs = {"anchor": x_data, 'neighbours': tf.gather(x_data, knns)}
labels = [np.ones(shape=(x_data.shape[0], kn)), np.ones(shape=(x_data.shape[0], kn))]

clustering_learner.compile(
    optimizer=keras.optimizers.AdamW(learning_rate=0.0005, weight_decay=0.0001),
    loss=losses,
    jit_compile=False,
)

clustering_learner.fit(
    x=inputs,
    y=labels,
    batch_size=512,
    epochs=50
)



Epoch 1/50


2025-05-30 11:15:59.550246: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:310] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.80GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


  1/118 ━━━━━━━━━━━━━━━━━━━━ 15:38 8s/step - clustering_loss: 0.0860 - loss: 0.9303 - simlarity_loss: 0.8444

2025-05-30 11:15:59.837096: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:310] Allocator (GPU_0_bfc) ran out of memory trying to allocate 5.64GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2025-05-30 11:15:59.895810: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:310] Allocator (GPU_0_bfc) ran out of memory trying to allocate 5.63GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


 56/118 ━━━━━━━━━━━━━━━━━━━━ 8s 130ms/step - clustering_loss: 0.1272 - loss: 0.6447 - simlarity_loss: 0.5175

In [16]:
clustering_probabilitiys = clustering_model.predict(x_data, batch_size=batch_size)
cluster_assigment = keras.ops.argmax(clustering_probabilitiys, axis=-1).numpy()

cluster_confidence = keras.ops.max(clustering_probabilitiys, axis=-1).numpy()

120/120 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step


In [17]:
from collections import defaultdict

clusters = defaultdict(list)

for idx, c in enumerate(cluster_assigment):
    clusters[c].append((idx, cluster_confidence[idx]))

non_empty_clusters = defaultdict(list)

for c in clusters.keys():
    if clusters[c]:
        non_empty_clusters[c] = clusters[c]

for c in range(num_clusters):
    print(f"Cluster {c}:  {len(clusters[c])}")

Cluster 0:  1488
Cluster 1:  1308
Cluster 2:  1889
Cluster 3:  1854
Cluster 4:  1234
Cluster 5:  1885
Cluster 6:  2097
Cluster 7:  1604
Cluster 8:  2051
Cluster 9:  2144
Cluster 10:  1499
Cluster 11:  1520
Cluster 12:  1299
Cluster 13:  1674
Cluster 14:  1299
Cluster 15:  1962
Cluster 16:  8275
Cluster 17:  1851
Cluster 18:  1806
Cluster 19:  1451
Cluster 20:  1832
Cluster 21:  1371
Cluster 22:  1771
Cluster 23:  8962
Cluster 24:  1626
Cluster 25:  2003
Cluster 26:  767
Cluster 27:  1478


In [18]:
from sklearn.neighbors import NearestNeighbors

k = 5
cluster_knns = {}

for c, members in non_empty_clusters.items():
    indices = [idx for idx, _ in members]
    if len(indices) < k + 1:

        continue

    clutser_features = feature_vectors[indices]
    nbrs = NearestNeighbors(n_neighbors=k + 1, algorithm='auto').fit(clutser_features)

    distances, n_indices = nbrs.kneighbors(clutser_features)

    knns = [ [indices[n] for n in n[1:]] for n in n_indices]

    cluster_knns[c] = dict(zip(indices, knns))



In [22]:
from scipy.sparse import lil_matrix, issparse, csr_matrix

num_nodes = feature_vectors.shape[0]
adjencency = lil_matrix((num_nodes, num_nodes))

for cluster in cluster_knns.values():
    for node_idx, knn_lisy in cluster.items():
        for neighbor in knn_lisy:
            adjencency[node_idx, neighbor] = 1
            adjencency[neighbor, node_idx] = 2

In [23]:
from scipy.sparse import csr_matrix, issparse

def graph_augmented(features, adjencency):
    if not issparse(adjencency):
        adj_aug = csr_matrix(adjencency)
    else:
        adj_aug = adjencency.copy()

    mask = np.random.rand(*adj_aug.shape) < 0.1
    mask_sparse = csr_matrix(mask)

    adj_aug = adj_aug.multiply(mask_sparse)

    feat_aug = features.copy()
    mask_features = np.random.rand(*feat_aug.shape) < 0.1
    feat_aug[~mask_features] = 0

    return adj_aug, feat_aug

In [26]:
generator = GraphDataGenerator(feature_vectors, adjencency, augmented_fn=graph_augmented)

In [28]:
feature_dims = feature_vectors.shape[0]
proj_dims = 64
output_dims = 64
hidden_dims = 128

encoder = create_gnn_encoder(feature_dims, hidden_dims, proj_dims)
projector = create_projector(proj_dims, output_dims)

model = SimSiamGNN(encoder, projector)


In [30]:
model.compile(
    optimizer=keras.optimizers.Adam(1e-3)
)

In [31]:
model.fit(generator, epochs=50)

In [32]:
model.save('first.keras')

In [37]:
train_labels = keras.utils.to_categorical(train_labels, num_classes=num_classes)

In [41]:
train_indices = tf.convert_to_tensor(train_indices, dtype=tf.int32)
train_labels = tf.convert_to_tensor(train_labels, dtype=tf.float32)

In [ ]:
model.save('completed_model.h5')